In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Classificacao").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/28 15:40:43 WARN Utils: Your hostname, LAPTOP-CO0U45G1, resolves to a loopback address: 127.0.1.1; using 172.20.235.103 instead (on interface eth0)
25/07/28 15:40:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/28 15:41:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [3]:
churn = spark.read.csv("/home/robson/curso_spark_pyspark_udemy/Churn.csv", header=True, inferSchema=True, sep=";")

In [4]:
churn.show(3)

+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|      0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1|8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8|1596608|            3|        1|             0|       11393157|     1|
+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
only showing top 3 rows


In [9]:
formula = RFormula(formula="Exited ~ .", featuresCol="features", labelCol="label", handleInvalid="skip")

In [10]:
churn_trans = formula.fit(churn).transform(churn).select("features", "label")

In [11]:
churn_trans.show(truncate=False)

+----------------------------------------------------------------+-----+
|features                                                        |label|
+----------------------------------------------------------------+-----+
|[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7]        |1.0  |
|[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7]  |0.0  |
|[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7]  |1.0  |
|(11,[0,1,4,5,7,10],[699.0,1.0,39.0,1.0,2.0,9382663.0])          |0.0  |
|[850.0,0.0,0.0,0.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0]   |0.0  |
|[645.0,0.0,0.0,1.0,44.0,8.0,1.1375578E7,2.0,1.0,0.0,1.4975671E7]|1.0  |
|[822.0,1.0,0.0,1.0,50.0,7.0,0.0,2.0,1.0,1.0,100628.0]           |0.0  |
|[376.0,0.0,1.0,0.0,29.0,4.0,1.1504674E7,4.0,1.0,0.0,1.1934688E7]|1.0  |
|[501.0,1.0,0.0,1.0,44.0,4.0,1.4205107E7,2.0,0.0,1.0,749405.0]   |0.0  |
|[684.0,1.0,0.0,1.0,27.0,2.0,1.3460388E7,1.0,1.0,1.0,7172573.0]  |0.0  |
|[528.0,1.0,0.0,1.0,31.0,6.0,1.0201672E7,2.0,0.0,0.

In [12]:
churnTreino, churnTeste = churn_trans.randomSplit([0.7, 0.3], seed=42)

In [13]:
print(churnTreino.count(), churnTeste.count())

7104 2896


In [14]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=5, seed=42)

In [15]:
modelo = dt.fit(churnTreino)

In [16]:
previsao = modelo.transform(churnTeste)

In [17]:
previsao.show(truncate=False)

+--------------------------------------------------------+-----+--------------+----------------------------------------+----------+
|features                                                |label|rawPrediction |probability                             |prediction|
+--------------------------------------------------------+-----+--------------+----------------------------------------+----------+
|(11,[0,1,3,4,7,10],[642.0,1.0,1.0,26.0,1.0,4747268.0])  |0.0  |[4482.0,505.0]|[0.8987367154601965,0.10126328453980349]|0.0       |
|(11,[0,1,4,5,7,10],[350.0,1.0,60.0,3.0,1.0,1.1379615E7])|1.0  |[41.0,258.0]  |[0.13712374581939799,0.862876254180602] |1.0       |
|(11,[0,1,4,5,7,10],[418.0,1.0,39.0,2.0,2.0,904171.0])   |0.0  |[4482.0,505.0]|[0.8987367154601965,0.10126328453980349]|0.0       |
|(11,[0,1,4,5,7,10],[449.0,1.0,21.0,7.0,2.0,1.7574392E7])|0.0  |[4482.0,505.0]|[0.8987367154601965,0.10126328453980349]|0.0       |
|(11,[0,1,4,5,7,10],[480.0,1.0,28.0,6.0,2.0,4813192.0])  |0.0  |[4482.0,505.

In [18]:
avaliar = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")

In [19]:
areaUnderROC = avaliar.evaluate(previsao)
print(f"Area Under ROC: {areaUnderROC}")

Area Under ROC: 0.6651656084399468
